In [ ]:
from core_functions import *

In [ ]:
(all_rgb_frames, all_gray_frames, all_kp, all_des) = get_kp_des("video_table.mov", points_count = 150)

In [ ]:
def get_max_score(cur_kp, cur_des, cur_points, scan_inds):
    max_score = 0
    ind = -1
    for j in range(len(scan_inds)):
        kp = all_kp[scan_inds[j]]
        des = all_des[scan_inds[j]]
        points = [(int(k.pt[0]), int(k.pt[1])) for k in kp]
        
        cur_h = find_H_mask_between_points(kp, des, cur_kp, cur_des)
        (cur_ind_from, cur_ind_to, m_from, m_to) = find_good_points_with_opt(cur_h, points, cur_points)
        first_j = float(np.count_nonzero(m_from)) / len(m_from)
        secound_j = float(np.count_nonzero(m_to)) / len(m_to)
        score_j = first_j + secound_j
        if score_j > max_score:
            max_score = score_j
            ind = j
            
    return (max_score, j)

In [ ]:
scan_inds = [0]
Hs = [np.eye(3)]

firts_point = (80, 400)
cur_point = (80, 400)
font = cv2.FONT_HERSHEY_SIMPLEX
x, y = 10, 20 #position of text 81 94 320
H_from_first = np.eye(3)

for i in range(1, 368):
    
    cur_frame = all_rgb_frames[i].copy()
    cur_frame = cv2.cvtColor(cur_frame, cv2.COLOR_BGR2RGB)

    prev_kp = all_kp[i-1]
    prev_des = all_des[i-1]
    prev_points = [(int(k.pt[0]), int(k.pt[1])) for k in prev_kp]
    
    cur_kp = all_kp[i]
    cur_des = all_des[i]
    cur_points = [(int(k.pt[0]), int(k.pt[1])) for k in cur_kp]
    
    cur_h = find_H_mask_between_points(prev_kp, prev_des, cur_kp, cur_des)
    H_from_first = np.dot(cur_h, H_from_first)
    
    (d, a) = get_min_distance(H_from_first, scan_inds, Hs)
    if (d > 200) or (a > 0.03):
        if (len(scan_inds) < 6):
            Hs.append(H_from_first.copy())
            scan_inds.append(i)

#     print(len(scan_inds))
    (cur_ind_from, cur_ind_to, m_from, m_to) = find_good_points_with_opt(cur_h, prev_points, cur_points)
    
    first = float(np.count_nonzero(m_from)) / len(m_from)
    secound = float(np.count_nonzero(m_to)) / len(m_to)
    score = first + secound
    max_score_str = "-"
    if (score < 0.7):
        (max_score, max_score_ind) = get_max_score(cur_kp, cur_des, cur_points, scan_inds)
        max_score_str = "max score {:1.2f} ind {} all {}".format(max_score, max_score_ind, len(scan_inds))
    
        if (max_score > (score + 0.8)):
            cur_h = find_H_mask_between_points(all_kp[max_score_ind], all_des[max_score_ind], cur_kp, cur_des)
            H_from_first = np.dot(cur_h, Hs[max_score_ind].copy())
        
#     first_str = " f {:1.2f} ".format(first)
#     secound_str = "s {:1.2f}".format(secound)
    score_str = "score {:1.2f} ind {}".format(score, i)
    
    
#     max_score,  = get_max_score()
    
    cur_point = np.array(homography_points_transform(H_from_first, [firts_point]))[0]
    cur_frame = cv2.circle(cur_frame, (int(cur_point[0]), int(cur_point[1])), 3, (255,255,255), 2)
    
    cv2.putText(cur_frame, score_str, (x,y), font, 0.75,(255,255,0), 2)
    cv2.putText(cur_frame, max_score_str, (x,y+20), font, 0.75,(255,255,0), 2)
    
    cv2.imshow('rgb', cur_frame)

    if cv2.waitKey(0) & 0xFF == ord('q'):
        break 